In [ ]:
import urllib.request
import pandas as pd
import json
from tinydb import TinyDB, Query
import pprint as pp

download_urls = [
'https://api.covidtracking.com/v2/us/daily.json'
]

for i in download_urls:
    webURL = urllib.request.urlopen(i)
    data = webURL.read()
    encoding = webURL.info().get_content_charset('utf-8')
    json_o=json.loads(data.decode(encoding))
#json_o is a python dictionary
#we need to flatten the data if we want to manipulate it using pandas


In [ ]:

meta_d=json_o['meta'] 

#meta_d is seperated on the meta key, since this is an info table there is little reason to manipulate it,
#but we can flatten it if we want to and then transpose it but I find it easier to just print the values

print(meta_d)

In [ ]:


#We have created a local NoSQL DB in the create_tinydb section
#normalise data based on feature key

df_data=pd.json_normalize(json_o,'data')
cols=list(df_data)

#convert to dataframe

df_data2= pd.DataFrame(df_data, columns=cols)

#Do something with data using dataframe
#fix nulls

df_data2.fillna(0, inplace=True)
               
#quick check
#df_data2.head(10)
                
Dict_data=df_data2.to_dict(orient='dict')

#convert back to dict and check for nulls
#pp.pprint(Dict_data)


In [ ]:

#insert in to nosqldb
db_meta = TinyDB('/home/nambu/Desktop/ETL pipeline/NOSQL/db_meta.json')
db_data1 = TinyDB('/home/nambu/Desktop/ETL pipeline/NOSQL/db_data1.json')
db_meta.insert(meta_d)
db_data1.insert(Dict_data)

In [ ]:
#ALTERNATIVES
import io

try:
    to_unicode = unicode
except NameError:
    to_unicode = str
with io.open('/home/nambu/Desktop/ETL pipeline/NOSQL/data.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(meta_d,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))
with io.open('/home/nambu/Desktop/ETL pipeline/NOSQL/data2.json', 'w', encoding='utf8') as outfile2:
    str_ = json.dumps(Dict_data,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile2.write(to_unicode(str_))


In [ ]:
json_file=df_data2.to_json(orient='columns')

with io.open('/home/nambu/Desktop/ETL pipeline/NOSQL/data3.json', 'w', encoding='utf8') as outfile3:
    outfile3.write(json_file)